In [1]:
import time
#time.sleep(60*60*3)
import os
os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"

In [2]:
import os
# --- Uncomment to use only CPU (e.g. GPU memory is too small)
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# import sys
# sys.path.append("/usr/local/cuda-10.1/bin")
# os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-10.1/lib64"

import tensorflow as tf
tf.test.is_gpu_available(cuda_only=True) 

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
# reate logger - nocer formatting
import logging

logger = logging.getLogger()
logging._warn_preinit_stderr = 0
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s: %(message)s')
ch = logging.StreamHandler()
ch.setFormatter(formatter)
logger.handlers = [ch]
logger.info("Logger started.")

2021-01-03 23:39:51,252: Logger started.


In [4]:
import sqlite3 
import numpy as np

def adapt_array(arr):
    """
    Save Numpy array to SqLite.
    Source:
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    """ 
    Load Numpy array from Sqlite.
    Source:
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)
data_path = os.path.join(os.getcwd(), "samplesGeneration", "prepared_data.bd")
data_path = "E:/tmp/data_2021_5.bd"
source = sqlite3.connect(data_path, detect_types=sqlite3.PARSE_DECLTYPES, check_same_thread=False)
db = sqlite3.connect(':memory:', detect_types=sqlite3.PARSE_DECLTYPES, check_same_thread=False)
source.backup(db)
source.close()

In [5]:
#import os
# --- Uncomment to use only CPU (e.g. GPU memory is too small)
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

# import sys
# sys.path.append("/usr/local/cuda-10.1/bin")
# os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-10.1/lib64"

#import tensorflow as tf
#tf.test.is_gpu_available(cuda_only=True) 

In [6]:
import io
sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)

db_cursor = db.cursor()
sql_query = "SELECT COALESCE(MAX(id)+1, 0) FROM data"
db_cursor.execute(sql_query)
number_of_samples = db_cursor.fetchone()[0]
print("number of samples: ", number_of_samples)

sql_query = "SELECT x_delta FROM data WHERE id == 1"
db_cursor.execute(sql_query)

number of samples:  1718361


In [7]:
import tensorflow.keras as keras


class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras from SQLite database'

    def __init__(self, db, indexes=None, batch_size=25,
                 shuffle=True, dim=(3, 39, 1), dim_y=(36, 1), y_name="y_delta"):
        'Initialization'
        if indexes is None:
            raise Exception("Indexes need to be provided!")
        sqlite3.register_adapter(np.ndarray, self.adapt_array)
        sqlite3.register_converter("array", self.convert_array)
        
        data_path = "E:/tmp/data_2021_5.bd"
        self.db = sqlite3.connect(data_path, detect_types=sqlite3.PARSE_DECLTYPES, check_same_thread=False)

        self.db.execute('pragma journal_mode=wal;')
        self.db.execute('pragma cache_size=1000000;')
        self.db.execute('pragma synchronous=0;')
        self.db.execute('pragma temp_store=MEMORY;')
        self.y_name = y_name
        self.dim = dim
        self.dim_y = dim_y
        self.db_cursor = self.db.cursor()
        self.N = len(indexes) - 1
        self.sample_index = indexes
        self.batch_size = int(batch_size)
        self.shuffle = shuffle
        self.on_epoch_end()

    def __del__(self):
        #pass
        self.db.close()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.N / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data - generates indexes of the batch'
        # Generate indexes of the batch
        samples_batch = np.arange((index) * self.batch_size, (index + 1) * self.batch_size)

        # Generate data
        while True:
            try:
                x_delta, x_abs_0, x_abs, x_force, y = self.__data_generation(samples_batch)
                break
            except ValueError:
                continue

        return ({"input_delta": x_delta, "input_abs": x_abs, "input_abs_0": x_abs_0, "input_force" : x_force}, y)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle:
            np.random.shuffle(self.sample_index)

    def __data_generation(self, samples_batch):
        'Generates data containing batch_size samples'  # X : (n_samples, *dim, n_channels)
        # Initialization
        x_delta = np.empty((self.batch_size, 4, 6))
        x_abs = np.empty((self.batch_size, 4, 9))
        x_abs_0 = np.empty((self.batch_size, 4, 9))
        x_force = np.empty((self.batch_size, 3))
        y = np.empty((self.batch_size, *self.dim_y))
        inds = self.sample_index[samples_batch]
        db_cursor = self.db_cursor
        sql_query = "SELECT x_delta, x_abs_0, x_abs, x_force, {y_name} FROM data WHERE id in ({index})". \
            format(y_name=self.y_name, index=','.join(str(ind) for ind in inds))
        db_cursor.execute(sql_query)
        try:
            for i, line in zip(range(self.batch_size), db_cursor.fetchall()):
                if line is None:
                    x_delta[i, :] = x_delta[i - 1, :]
                    x_abs_0[i, :] = x_abs_0[i - 1, :]
                    x_abs[i, :] = x_abs[i - 1, :]
                    x_force[i, :] = x_force[i - 1, :]
                    y[i] = y[i - 1]
                    continue  # Bad, temporary solution
                x_delta[i, :] = np.array(line[0])
                x_abs_0[i, :] = np.array(line[1])
                x_abs[i, :] = np.array(line[2])
                x_force[i, :] = np.array(line[3])
                y[i, :] = np.array(line[4])
        except TypeError as e:
            print(sql_query)
            raise e
        return x_delta, x_abs_0, x_abs, x_force, y

    def adapt_array(self, arr):
        """
        Save Numpy array to SqLite.
        Source:
        http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
        """
        out = io.BytesIO()
        np.save(out, arr)
        out.seek(0)
        return sqlite3.Binary(out.read())

    def convert_array(self, text):
        """
        Load Numpy array from Sqlite.
        Source:
        http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
        """
        out = io.BytesIO(text)
        out.seek(0)
        return np.load(out)


test_train_ratio = 0.1

test_size = int(test_train_ratio * number_of_samples)

indexes = np.arange(0, number_of_samples - 1)
np.random.shuffle(indexes)
train_indexes, val_indexes = indexes[:-test_size], indexes[test_size:]

training_generator = DataGenerator(db, indexes=train_indexes, batch_size=8, dim=(3, 39), dim_y=(4, 6))
validation_generator = DataGenerator(db, indexes=val_indexes, batch_size=8, dim=(3, 39), dim_y=(4, 6))

In [8]:
import time
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=2)


models_dir = os.path.join(os.getcwd(), "samplesGeneration", "logs")

In [9]:
validation_generator[0]

({'input_delta': array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],
  
         [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
            0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
            0.00000000e+00,  0.00000000e+00,

In [10]:
# Try learning for all of verts:

verts = range(14)
pivot_vert = 3

from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Concatenate
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.optimizers import RMSprop
from collections import deque
from tensorflow.keras.callbacks import EarlyStopping
import time
import tensorflow.keras.backend as K


def special_loss(y_true, y_pred):
    custom_loss=tf.add(tf.square(y_true-y_pred)/tf.add(tf.square(y_pred), 0.1), tf.keras.losses.MSE(y_true, y_pred))
    return custom_loss    

def loss(y_true,y_pred):
    return K.sum(K.square(y_pred - y_true) * K.square(100 * y_true) + K.square(y_pred - y_true))

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
def create_and_train_network(generator, val_generator, y_name="y_delta", y_shape=(4, 6), model_id=0, epohs=10):
    log_dir = os.path.join(os.getcwd(), "samplesGeneration", "logs", "model_{}_{}_{}_{}".format(y_name, y_shape, model_id, time.time()))
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    training_generator = DataGenerator(db, indexes=train_indexes, batch_size=8, dim=(3, 39), dim_y=y_shape, y_name=y_name)
    validation_generator = DataGenerator(db, indexes=val_indexes, batch_size=8, dim=(3, 39), dim_y=y_shape, y_name=y_name)

    best_model_file = "{}_{}_{}_model.h5".format(y_name, y_shape, model_id)
    best_model = ModelCheckpoint(best_model_file, monitor='val_loss', mode='min',verbose=1, save_best_only=True)
    
    class CustomSaveCallback(keras.callbacks.Callback):

        def on_test_begin(self, logs=None):
            self.model.save("model\\model_{}_{}_{}_{}".format(y_name, y_shape, model_id, time.time()), save_format='tf')
    my_cb = CustomSaveCallback()
    
    def create_model0():
        input_delta = Input(shape=(4, 6), name='input_delta')
        input_abs = Input(shape=(4, 9), name='input_abs')
        input_force = Input(shape=(3, 1), name='input_force')
        
        input_delta_flat = Flatten()(input_delta)
        input_abs_flat = Flatten()(input_abs)
        input_force_flat = Flatten()(input_force)
        
        x = Concatenate()([input_delta_flat, input_abs_flat, input_force_flat])
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(np.prod(y_shape))(x)
        
        output_1 = Reshape(y_shape)(x)
        model = Model(inputs=[input_delta, input_abs, input_force], outputs=[output_1])
        model.compile(loss="mse", optimizer='rmsprop')
        return model
    
    def create_model1():
        input_delta = Input(shape=(4, 6), name='input_delta')
        input_abs = Input(shape=(4, 9), name='input_abs')
        input_force = Input(shape=(3, 1), name='input_force')
        
        input_delta_flat = Flatten()(input_delta)
        input_abs_flat = Flatten()(input_abs)
        input_force_flat = Flatten()(input_force)

        x = Dense(64, activation='relu')(input_force_flat)
        x = Dropout(0.2)(x)
        x = Concatenate()([x, input_delta_flat])
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Concatenate()([x, input_abs_flat])
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(np.prod(y_shape))(x)
        
        output_1 = Reshape(y_shape)(x)
        model = Model(inputs=[input_delta, input_abs, input_force], outputs=[output_1])
        model.compile(loss="mse", optimizer='rmsprop')
        return model
    
    def create_model2():
        input_delta = Input(shape=(4, 6), name='input_delta')
        input_abs = Input(shape=(4, 9), name='input_abs')
        input_force = Input(shape=(3, 1), name='input_force')
        
        input_delta_flat = Flatten()(input_delta)
        input_abs_flat = Flatten()(input_abs)
        input_force_flat = Flatten()(input_force)
        
        x = Concatenate()([input_delta_flat, input_abs_flat, input_force_flat])
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(np.prod(y_shape))(x)
        
        output_1 = Reshape(y_shape)(x)
        model = Model(inputs=[input_delta, input_abs, input_force], outputs=[output_1])
        model.compile(loss="mse", optimizer='rmsprop')
        return model
    
    def create_model3():
        input_delta = Input(shape=(4, 6), name='input_delta')
        input_abs_0 = Input(shape=(4, 9), name='input_abs_0')
        input_abs = Input(shape=(4, 9), name='input_abs')
        input_force = Input(shape=(3, 1), name='input_force')

        input_abs_0_flat = Flatten()(input_abs_0)
        input_abs_flat = Flatten()(input_abs)
        input_force_flat = Flatten()(input_force)
        x = Concatenate()([input_abs_0_flat, input_abs_flat])
        x = Reshape((2, 4*9))(x)
        x = LSTM(np.prod(generator.dim_y), return_sequences=False, dropout=0.2, 
        activation="tanh", recurrent_activation="sigmoid", recurrent_dropout=0, unroll=False, use_bias=True
                )(x)
        x = Dense(np.prod(y_shape))(x)
        
        output_1 = Reshape(y_shape)(x)
        model = Model(inputs=[input_abs_0, input_abs, input_force], outputs=[output_1])
        model.compile(loss="mse", optimizer='rmsprop')
        return model
    
    def create_model4():
        input_delta = Input(shape=(4, 6), name='input_delta')
        input_abs_0 = Input(shape=(4, 9), name='input_abs_0')
        input_abs = Input(shape=(4, 9), name='input_abs')
        input_force = Input(shape=(3, 1), name='input_force')

        input_abs_0_flat = Flatten()(input_abs_0)
        input_abs_flat = Flatten()(input_abs)
        input_force_flat = Flatten()(input_force)
        x = Concatenate()([input_abs_0_flat, input_abs_flat])
        x = Reshape((2, 4*9))(x)
        x = GRU(np.prod(generator.dim_y), return_sequences=False, 
        activation="tanh", recurrent_activation="sigmoid", recurrent_dropout=0, unroll=False, use_bias=True, reset_after=True
        )(x)
        x = Flatten()(x)
        x = Concatenate()([x, input_force_flat])
        x = Dropout(0.2)(x)
        x = Dense(np.prod(y_shape))(x)
        
        output_1 = Reshape(y_shape)(x)
        model = Model(inputs=[input_abs_0, input_abs, input_force], outputs=[output_1])
        model.compile(loss="mse", optimizer='rmsprop')
        return model

    logger.info("Create model...")
    if model_id == 0:
        model = create_model0()
    elif model_id == 1:
        model = create_model1()
    elif model_id == 2:
        model = create_model2()
    elif model_id == 3:
        model = create_model3()
    elif model_id == 4:
        model = create_model4()
    model.summary()
    logger.info("Fit model..")
    network = model.fit(training_generator, verbose=1, validation_data=validation_generator, 
                                  epochs=epohs, callbacks=[my_cb, early_stopping, best_model, tensorboard_callback])
    logger.info("Summary")
    model.summary()
    
    models_dir = os.path.join(os.getcwd(), "samplesGeneration", "logs", "model_{}".format(time.time()))
    model.save(os.path.join(models_dir, "model"))
    return network

logger.info("start")

#training_generator = tf.data.Dataset.from_generator(lambda: training_generator,          
#                                                    output_types=(tf.float32, tf.float32))
#validation_generator = tf.data.Dataset.from_generator(lambda: training_generator,          
#                                                    output_types=(tf.float32, tf.float32))

network = create_and_train_network(training_generator, validation_generator, "y_delta", (4, 6))
network = create_and_train_network(training_generator, validation_generator, "y_delta", (4, 6), 1)
network = create_and_train_network(training_generator, validation_generator, "y_delta", (4, 6), 2)
network = create_and_train_network(training_generator, validation_generator, "y_delta", (4, 6), 3)
network = create_and_train_network(training_generator, validation_generator, "y_delta", (4, 6), 4)
network2 = create_and_train_network(training_generator, validation_generator, "y_abs", (4, 9), 0)
network2 = create_and_train_network(training_generator, validation_generator, "y_abs", (4, 9), 1)
network2 = create_and_train_network(training_generator, validation_generator, "y_abs", (4, 9), 2)

2021-01-03 23:39:59,666: start
2021-01-03 23:40:00,004: Create model...
2021-01-03 23:40:01,441: Fit model..


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_delta (InputLayer)        [(None, 4, 6)]       0                                            
__________________________________________________________________________________________________
input_abs (InputLayer)          [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
input_force (InputLayer)        [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
flatten (Flatten)               (None, 24)           0           input_delta[0][0]                
______________________________________________________________________________________________

D:\ProgramData\Anaconda3\envs\tf38\lib\site-packages\tensorflow\python\keras\engine\functional.py:594: UserWarning: Input dict contained keys ['input_abs_0'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


     3/193315 [..............................] - ETA: 24:41:30 - loss: 1.6185WARNING:tensorflow:Callback method `on_train_batch_begin` is slow compared to the batch time (batch time: 0.0178s vs `on_train_batch_begin` time: 0.0217s). Check your callbacks.


2021-01-03 23:40:07,749: Callback method `on_train_batch_begin` is slow compared to the batch time (batch time: 0.0178s vs `on_train_batch_begin` time: 0.0217s). Check your callbacks.


...

2021-01-05 07:03:23,652: Assets written to: model\model_y_abs_(4, 9)_2_1609826601.9458225\assets
D:\ProgramData\Anaconda3\envs\tf38\lib\site-packages\tensorflow\python\keras\engine\functional.py:594: UserWarning: Input dict contained keys ['input_abs_0'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


193315/193315 [==============================] - 2968s 15ms/step - loss: 0.3040 - val_loss: 0.0499

Epoch 00001: val_loss improved from inf to 0.04993, saving model to y_abs_(4, 9)_2_model.h5
Epoch 2/10
193310/193315 [============================>.] - ETA: 0s - loss: 0.2151INFO:tensorflow:Assets written to: model\model_y_abs_(4, 9)_2_1609829548.4126158\assets


2021-01-05 07:52:29,972: Assets written to: model\model_y_abs_(4, 9)_2_1609829548.4126158\assets


193315/193315 [==============================] - 2949s 15ms/step - loss: 0.2151 - val_loss: 0.0480

Epoch 00002: val_loss improved from 0.04993 to 0.04795, saving model to y_abs_(4, 9)_2_model.h5
Epoch 3/10
109087/193315 [===============>..............] - ETA: 10:59 - loss: 0.2333

KeyboardInterrupt: 

In [13]:
# Try learning for all of verts:

verts = range(14)
pivot_vert = 3

from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Concatenate
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.optimizers import RMSprop
from collections import deque
from tensorflow.keras.callbacks import EarlyStopping
import time
import tensorflow.keras.backend as K


def special_loss(y_true, y_pred):
    custom_loss=tf.add(tf.square(y_true-y_pred)/tf.add(tf.square(y_pred), 0.1), tf.keras.losses.MSE(y_true, y_pred))
    return custom_loss    

def loss(y_true,y_pred):
    return K.sum(K.square(y_pred - y_true) * K.square(100 * y_true) + K.square(y_pred - y_true))

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
def create_and_train_network(generator, val_generator, y_name="y_delta", y_shape=(4, 6), model_id=0, epohs=10):
    log_dir = os.path.join(os.getcwd(), "samplesGeneration", "logs", "model_{}_{}_{}_{}".format(y_name, y_shape, model_id, time.time()))
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    training_generator = DataGenerator(db, indexes=train_indexes, batch_size=8, dim=(3, 39), dim_y=y_shape, y_name=y_name)
    validation_generator = DataGenerator(db, indexes=val_indexes, batch_size=8, dim=(3, 39), dim_y=y_shape, y_name=y_name)

    best_model_file = "{}_{}_{}_model.h5".format(y_name, y_shape, model_id)
    best_model = ModelCheckpoint(best_model_file, monitor='val_loss', mode='min',verbose=1, save_best_only=True)
    
    class CustomSaveCallback(keras.callbacks.Callback):

        def on_test_begin(self, logs=None):
            self.model.save("model\\model_{}_{}_{}_{}".format(y_name, y_shape, model_id, time.time()), save_format='tf')
    my_cb = CustomSaveCallback()
    
    def create_model0():
        input_delta = Input(shape=(4, 6), name='input_delta')
        input_abs = Input(shape=(4, 9), name='input_abs')
        input_force = Input(shape=(3, 1), name='input_force')
        
        input_delta_flat = Flatten()(input_delta)
        input_abs_flat = Flatten()(input_abs)
        input_force_flat = Flatten()(input_force)
        
        x = Concatenate()([input_delta_flat, input_abs_flat, input_force_flat])
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(np.prod(y_shape))(x)
        
        output_1 = Reshape(y_shape)(x)
        model = Model(inputs=[input_delta, input_abs, input_force], outputs=[output_1])
        model.compile(loss="mse", optimizer='rmsprop')
        return model
    
    def create_model1():
        input_delta = Input(shape=(4, 6), name='input_delta')
        input_abs = Input(shape=(4, 9), name='input_abs')
        input_force = Input(shape=(3, 1), name='input_force')
        
        input_delta_flat = Flatten()(input_delta)
        input_abs_flat = Flatten()(input_abs)
        input_force_flat = Flatten()(input_force)

        x = Dense(64, activation='relu')(input_force_flat)
        x = Dropout(0.2)(x)
        x = Concatenate()([x, input_delta_flat])
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Concatenate()([x, input_abs_flat])
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(np.prod(y_shape))(x)
        
        output_1 = Reshape(y_shape)(x)
        model = Model(inputs=[input_delta, input_abs, input_force], outputs=[output_1])
        model.compile(loss="mse", optimizer='rmsprop')
        return model
    
    def create_model2():
        input_delta = Input(shape=(4, 6), name='input_delta')
        input_abs = Input(shape=(4, 9), name='input_abs')
        input_force = Input(shape=(3, 1), name='input_force')
        
        input_delta_flat = Flatten()(input_delta)
        input_abs_flat = Flatten()(input_abs)
        input_force_flat = Flatten()(input_force)
        
        x = Concatenate()([input_delta_flat, input_abs_flat, input_force_flat])
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Dense(np.prod(y_shape))(x)
        
        output_1 = Reshape(y_shape)(x)
        model = Model(inputs=[input_delta, input_abs, input_force], outputs=[output_1])
        model.compile(loss="mse", optimizer='rmsprop')
        return model
    
    def create_model3():
        input_delta = Input(shape=(4, 6), name='input_delta')
        input_abs_0 = Input(shape=(4, 9), name='input_abs_0')
        input_abs = Input(shape=(4, 9), name='input_abs')
        input_force = Input(shape=(3, 1), name='input_force')

        input_abs_0_flat = Flatten()(input_abs_0)
        input_abs_flat = Flatten()(input_abs)
        input_force_flat = Flatten()(input_force)
        x = Concatenate()([input_abs_0_flat, input_abs_flat])
        x = Reshape((2, 4*9))(x)
        x = LSTM(np.prod(generator.dim_y), return_sequences=False, dropout=0.2, 
        activation="tanh", recurrent_activation="sigmoid", recurrent_dropout=0, unroll=False, use_bias=True
                )(x)
        x = Dense(np.prod(y_shape))(x)
        
        output_1 = Reshape(y_shape)(x)
        model = Model(inputs=[input_abs_0, input_abs, input_force], outputs=[output_1])
        model.compile(loss="mse", optimizer='rmsprop')
        return model
    
    def create_model4():
        input_delta = Input(shape=(4, 6), name='input_delta')
        input_abs_0 = Input(shape=(4, 9), name='input_abs_0')
        input_abs = Input(shape=(4, 9), name='input_abs')
        input_force = Input(shape=(3, 1), name='input_force')

        input_abs_0_flat = Flatten()(input_abs_0)
        input_abs_flat = Flatten()(input_abs)
        input_force_flat = Flatten()(input_force)
        x = Concatenate()([input_abs_0_flat, input_abs_flat])
        x = Reshape((2, 4*9))(x)
        x = GRU(np.prod(generator.dim_y), return_sequences=False, 
        activation="tanh", recurrent_activation="sigmoid", recurrent_dropout=0, unroll=False, use_bias=True, reset_after=True
        )(x)
        x = Flatten()(x)
        x = Concatenate()([x, input_force_flat])
        x = Dropout(0.2)(x)
        x = Dense(np.prod(y_shape))(x)
        
        output_1 = Reshape(y_shape)(x)
        model = Model(inputs=[input_abs_0, input_abs, input_force], outputs=[output_1])
        model.compile(loss="mse", optimizer='rmsprop')
        return model

    logger.info("Create model...")
    if model_id == 0:
        model = create_model0()
    elif model_id == 1:
        model = create_model1()
    elif model_id == 2:
        model = create_model2()
    elif model_id == 3:
        model = create_model3()
    elif model_id == 4:
        model = create_model4()
    model.summary()
    logger.info("Fit model..")
    network = model.fit(training_generator, verbose=1, validation_data=validation_generator, 
                                  epochs=epohs, callbacks=[my_cb, early_stopping, best_model, tensorboard_callback])
    logger.info("Summary")
    model.summary()
    
    models_dir = os.path.join(os.getcwd(), "samplesGeneration", "logs", "model_{}".format(time.time()))
    model.save(os.path.join(models_dir, "model"))
    return network


network2 = create_and_train_network(training_generator, validation_generator, "y_abs", (4, 9), 4, epohs=2)
network2 = create_and_train_network(training_generator, validation_generator, "y_abs", (4, 9), 3, epohs=2)

2021-01-05 08:32:39,228: Create model...
2021-01-05 08:32:39,428: Fit model..


Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_abs_0 (InputLayer)        [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
input_abs (InputLayer)          [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
flatten_32 (Flatten)            (None, 36)           0           input_abs_0[0][0]                
__________________________________________________________________________________________________
flatten_33 (Flatten)            (None, 36)           0           input_abs[0][0]                  
____________________________________________________________________________________________

2021-01-05 08:32:43,092: Callback method `on_train_batch_begin` is slow compared to the batch time (batch time: 0.0138s vs `on_train_batch_begin` time: 0.0457s). Check your callbacks.


2021-01-05 08:32:43,096: Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0138s vs `on_train_batch_end` time: 0.2594s). Check your callbacks.


193312/193315 [============================>.] - ETA: 0s - loss: 0.2754

...

2021-01-05 11:49:17,552: Summary



Epoch 00002: val_loss did not improve from 0.15657
Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_abs_0 (InputLayer)        [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
input_abs (InputLayer)          [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
flatten_36 (Flatten)            (None, 36)           0           input_abs_0[0][0]                
__________________________________________________________________________________________________
flatten_37 (Flatten)            (None, 36)           0           input_abs[0][0]                  
_______________________________________

2021-01-05 11:49:19,532: Found untraced functions such as lstm_cell_2_layer_call_fn, lstm_cell_2_layer_call_and_return_conditional_losses, lstm_cell_2_layer_call_fn, lstm_cell_2_layer_call_and_return_conditional_losses, lstm_cell_2_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: D:\git\ueflow\samplesGeneration\logs\model_1609843757.5545084\model\assets


2021-01-05 11:49:21,189: Assets written to: D:\git\ueflow\samplesGeneration\logs\model_1609843757.5545084\model\assets


In [14]:
create_and_train_network(training_generator, validation_generator, "y_abs", (4, 9), 4, epohs=5)
create_and_train_network(training_generator, validation_generator, "y_abs", (4, 9), 3, epohs=5)

2021-01-05 12:00:59,816: Create model...
2021-01-05 12:01:00,032: Fit model..


Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_abs_0 (InputLayer)        [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
input_abs (InputLayer)          [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
flatten_39 (Flatten)            (None, 36)           0           input_abs_0[0][0]                
__________________________________________________________________________________________________
flatten_40 (Flatten)            (None, 36)           0           input_abs[0][0]                  
___________________________________________________________________________________________

D:\ProgramData\Anaconda3\envs\tf38\lib\site-packages\tensorflow\python\keras\engine\functional.py:594: UserWarning: Input dict contained keys ['input_delta'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


     3/193315 [..............................] - ETA: 71:09:54 - loss: 5.6818WARNING:tensorflow:Callback method `on_train_batch_begin` is slow compared to the batch time (batch time: 0.0075s vs `on_train_batch_begin` time: 0.0731s). Check your callbacks.


2021-01-05 12:01:04,136: Callback method `on_train_batch_begin` is slow compared to the batch time (batch time: 0.0075s vs `on_train_batch_begin` time: 0.0731s). Check your callbacks.


...

2021-01-05 17:43:32,783: Summary


Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_abs_0 (InputLayer)        [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
input_abs (InputLayer)          [(None, 4, 9)]       0                                            
__________________________________________________________________________________________________
flatten_43 (Flatten)            (None, 36)           0           input_abs_0[0][0]                
__________________________________________________________________________________________________
flatten_44 (Flatten)            (None, 36)           0           input_abs[0][0]                  
___________________________________________________________________________________________

2021-01-05 17:43:34,508: Found untraced functions such as lstm_cell_3_layer_call_fn, lstm_cell_3_layer_call_and_return_conditional_losses, lstm_cell_3_layer_call_fn, lstm_cell_3_layer_call_and_return_conditional_losses, lstm_cell_3_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: D:\git\ueflow\samplesGeneration\logs\model_1609865012.7851317\model\assets


2021-01-05 17:43:36,113: Assets written to: D:\git\ueflow\samplesGeneration\logs\model_1609865012.7851317\model\assets


In [ ]:
model = tf.keras.models.load_model(best_model_file, custom_objects={"loss": loss})

model.save("model\\improved_model_wip_abs2", save_format='tf')

In [ ]:
saver = tf.train.Saver()

In [ ]:
import io

sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter("array", convert_array)
db = sqlite3.connect(data_path, detect_types=sqlite3.PARSE_DECLTYPES)
db_cursor = db.cursor()
sql_query = "SELECT x, y FROM data WHERE id == 1"
db_cursor.execute(sql_query)
data = db_cursor.fetchone()[0]
print(data.shape)
predicted= list()
frames = np.array(np.array_split(data, 3))
for f in frames:
    predicted.append(f)

for n in range(240):
    _in  = np.concatenate((predicted[-3], predicted[-2], predicted[-1])).reshape(116, 1)
    pred = model.predict(_in)
    p = (n ==5) *10
    np.append(pred, p)
    np.append(pred, data[-2])
    np.append(pred, data[-1])
    print(pred.shape)
    predicted.append(pred)

In [ ]:
import matplotlib as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d', xlim=(-2,2),ylim=(-2,2), zlim=(0, 10))

xs = list()
ys = list()
zs = list()
sizes = list()
sample_to_plot = data[600]
for num, f in enumerate(sample_to_plot):
    for i in range(4):
        v = [f[9*i], f[9*i + 1], f[9*i + 2]]
        # print(v)
        xs.append(v[0])
        ys.append(v[2])
        zs.append(v[1])
        sizes.append([(len(sample_to_plot) - float(num))/len(sample_to_plot), 0, 0])
ax.scatter(xs, ys, zs, c=sizes)

plt.show()